# Dynamic Attribution and Properties

In [1]:
import json

with open('data/osconfeed.json') as fp:
    feed = json.load(fp)

In [2]:
sorted(feed['Schedule'].keys())

['conferences', 'events', 'speakers', 'venues']

In [5]:
for key, value in sorted(feed['Schedule'].items()):
    print(f'{len(value): 3}\t{key}')

  1	conferences
 484	events
 357	speakers
 53	venues


In [6]:
feed['Schedule']['speakers'][-1]['name']

'Carina C. Zona'

---

In [25]:
import keyword
from collections import abc


class FrozenJSON:
    """ A read-only facade for navigating a JSON-like object
        using attribute natation
    """

    # As a class method, the first argument `__new__` gets is the class itself
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            # The default behavior is to delegate to the `__new__` of a superclass. 
            return super().__init__(cls)
        # The remaining lines of `__new__` are exactly as in the old `build` method. 
        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]
        return arg

    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping:
            if keyword.iskeyword(key):
                key += '_'
            self.__data[key] = value
    
    def __getattr__(self, name):
        try:
            return getattr(self.__data, name)
        except AttributeError:
        # Now we just call the `FrozenJSON` class, which Python handles by calling `FrozenJSON.__new__`. 
            return FrozenJSON.build(self.__data[name])
    
    def __dir__(self):
        return self.__data.keys()
    
    @classmethod
    def build(cls, obj):
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            return [cls.build(item) for item in obj]
        return obj

In [26]:
feed_json = FrozenJSON(feed)

feed_json.keys()

dict_keys(['Schedule'])

In [19]:
len(feed_json.Schedule.speakers)

357

In [20]:
talk = feed_json.Schedule.events[40]
talk

In [21]:
talk.name

'There *Will* Be Bugs'

In [28]:
try:
    print(talk.flavor)
except KeyError:
    print(f'Trying to read a missing attribute. ')

Trying to read a missing attribute. 


---

In [29]:
import json

JSON_PATH = 'data/osconfeed.json'


class Record:

    def __init__(self, **kwargs) -> None:
        self.__dict__.update(kwargs)
    
    def __repr__(self):
        return f'<{self.__class__.__name__} serial={self.serial!r}>'
    

def load(path=JSON_PATH):
    records = {}
    with open(path) as fp:
        raw_data = json.load(fp)
    for collection, raw_records in raw_data['Schedule'].items():
        record_type = collection[:-1]
        for raw_record in raw_records:
            key = f'{record_type}.{raw_record["serial"]}'
            records[key] = Record(**raw_record)
    return records

In [34]:
records = load()
speaker = records['speaker.3471']
speaker

<Record serial=3471>

In [31]:
speaker.name, speaker.twitter

('Anna Martelli Ravenscroft', 'annaraven')